In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import random
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90., high=90., size=2000)
lngs = np.random.uniform(low=-180., high=180., size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

740

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14d496981f3ab4ab4fbc15adae63ab96


In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_current = city_weather["weather"]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_current})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | atuona
City not found. Skipping...
Processing Record 2 of Set 1 | vaini
City not found. Skipping...
Processing Record 3 of Set 1 | teya
City not found. Skipping...
Processing Record 4 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 5 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 6 of Set 1 | avarua
City not found. Skipping...
Processing Record 7 of Set 1 | kodiak
City not found. Skipping...
Processing Record 8 of Set 1 | lebu
City not found. Skipping...
Processing Record 9 of Set 1 | husavik
City not found. Skipping...
Processing Record 10 of Set 1 | mataura
City not found. Skipping...
Processing Record 11 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 12 of Set 1 | umzimvubu
City not found. Skipping...
Processing Record 13 of Set 1 | sitka
City not found. Skipping...
Processing Record 14 of Set 1 | bastia
City not found. Skippi

City not found. Skipping...
Processing Record 21 of Set 3 | yellowknife
City not found. Skipping...
Processing Record 22 of Set 3 | borovskoy
City not found. Skipping...
Processing Record 23 of Set 3 | vardo
City not found. Skipping...
Processing Record 24 of Set 3 | sao joao da barra
City not found. Skipping...
Processing Record 25 of Set 3 | alyangula
City not found. Skipping...
Processing Record 26 of Set 3 | bumbesti-jiu
City not found. Skipping...
Processing Record 27 of Set 3 | tuktoyaktuk
City not found. Skipping...
Processing Record 28 of Set 3 | port elizabeth
City not found. Skipping...
Processing Record 29 of Set 3 | luderitz
City not found. Skipping...
Processing Record 30 of Set 3 | merauke
City not found. Skipping...
Processing Record 31 of Set 3 | provideniya
City not found. Skipping...
Processing Record 32 of Set 3 | sataua
City not found. Skipping...
Processing Record 33 of Set 3 | bethel
City not found. Skipping...
Processing Record 34 of Set 3 | saint-augustin
City n

City not found. Skipping...
Processing Record 39 of Set 5 | snasa
City not found. Skipping...
Processing Record 40 of Set 5 | campo maior
City not found. Skipping...
Processing Record 41 of Set 5 | kalispell
City not found. Skipping...
Processing Record 42 of Set 5 | piste
City not found. Skipping...
Processing Record 43 of Set 5 | northam
City not found. Skipping...
Processing Record 44 of Set 5 | havelock
City not found. Skipping...
Processing Record 45 of Set 5 | paraiso
City not found. Skipping...
Processing Record 46 of Set 5 | brae
City not found. Skipping...
Processing Record 47 of Set 5 | north bend
City not found. Skipping...
Processing Record 48 of Set 5 | puerto ayora
City not found. Skipping...
Processing Record 49 of Set 5 | pemangkat
City not found. Skipping...
Processing Record 50 of Set 5 | meulaboh
City not found. Skipping...
Processing Record 1 of Set 6 | nador
City not found. Skipping...
Processing Record 2 of Set 6 | ambon
City not found. Skipping...
Processing Reco

City not found. Skipping...
Processing Record 7 of Set 8 | taoudenni
City not found. Skipping...
Processing Record 8 of Set 8 | baykit
City not found. Skipping...
Processing Record 9 of Set 8 | flinders
City not found. Skipping...
Processing Record 10 of Set 8 | saravan
City not found. Skipping...
Processing Record 11 of Set 8 | clyde river
City not found. Skipping...
Processing Record 12 of Set 8 | sabang
City not found. Skipping...
Processing Record 13 of Set 8 | udaipura
City not found. Skipping...
Processing Record 14 of Set 8 | coihaique
City not found. Skipping...
Processing Record 15 of Set 8 | saint-louis
City not found. Skipping...
Processing Record 16 of Set 8 | deputatskiy
City not found. Skipping...
Processing Record 17 of Set 8 | saint george
City not found. Skipping...
Processing Record 18 of Set 8 | porto belo
City not found. Skipping...
Processing Record 19 of Set 8 | borogontsy
City not found. Skipping...
Processing Record 20 of Set 8 | valleyview
City not found. Skipp

City not found. Skipping...
Processing Record 25 of Set 10 | bhindar
City not found. Skipping...
Processing Record 26 of Set 10 | bandarbeyla
City not found. Skipping...
Processing Record 27 of Set 10 | ca mau
City not found. Skipping...
Processing Record 28 of Set 10 | crab hill
City not found. Skipping...
Processing Record 29 of Set 10 | kawambwa
City not found. Skipping...
Processing Record 30 of Set 10 | ust-nera
City not found. Skipping...
Processing Record 31 of Set 10 | arlit
City not found. Skipping...
Processing Record 32 of Set 10 | sayyan
City not found. Skipping...
Processing Record 33 of Set 10 | iskateley
City not found. Skipping...
Processing Record 34 of Set 10 | vostok
City not found. Skipping...
Processing Record 35 of Set 10 | la rioja
City not found. Skipping...
Processing Record 36 of Set 10 | benjamin constant
City not found. Skipping...
Processing Record 37 of Set 10 | launceston
City not found. Skipping...
Processing Record 38 of Set 10 | sabak
City not found. S

City not found. Skipping...
Processing Record 42 of Set 12 | maniitsoq
City not found. Skipping...
Processing Record 43 of Set 12 | bileca
City not found. Skipping...
Processing Record 44 of Set 12 | sechura
City not found. Skipping...
Processing Record 45 of Set 12 | satitoa
City not found. Skipping...
Processing Record 46 of Set 12 | trelew
City not found. Skipping...
Processing Record 47 of Set 12 | saint-felicien
City not found. Skipping...
Processing Record 48 of Set 12 | victoria point
City not found. Skipping...
Processing Record 49 of Set 12 | harrismith
City not found. Skipping...
Processing Record 50 of Set 12 | karlshamn
City not found. Skipping...
Processing Record 1 of Set 13 | maarianhamina
City not found. Skipping...
Processing Record 2 of Set 13 | honningsvag
City not found. Skipping...
Processing Record 3 of Set 13 | korla
City not found. Skipping...
Processing Record 4 of Set 13 | timra
City not found. Skipping...
Processing Record 5 of Set 13 | hvammstangi
City not f

City not found. Skipping...
Processing Record 9 of Set 15 | garhi khairo
City not found. Skipping...
Processing Record 10 of Set 15 | terney
City not found. Skipping...
Processing Record 11 of Set 15 | pombas
City not found. Skipping...
Processing Record 12 of Set 15 | tezu
City not found. Skipping...
Processing Record 13 of Set 15 | jiwani
City not found. Skipping...
Processing Record 14 of Set 15 | grand forks
City not found. Skipping...
Processing Record 15 of Set 15 | logan
City not found. Skipping...
Processing Record 16 of Set 15 | kupang
City not found. Skipping...
Processing Record 17 of Set 15 | shimoda
City not found. Skipping...
Processing Record 18 of Set 15 | edd
City not found. Skipping...
Processing Record 19 of Set 15 | adrar
City not found. Skipping...
Processing Record 20 of Set 15 | abu samrah
City not found. Skipping...
Processing Record 21 of Set 15 | sibanicu
City not found. Skipping...
Processing Record 22 of Set 15 | utiroa
City not found. Skipping...
Processing

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

""


In [9]:
city_data_df.to_csv('WeatherPy_Database.csv')